In [2]:
# !pip install pymysql
# !pip install pymssql

import LibData
import LibUtils
import LibML
import LibFile
import LibEE

import pandas as pd
import numpy as np


In [4]:
#for sfs
# !pip install mlxtend
from mlxtend.feature_selection import SequentialFeatureSelector
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor

def Selection(x, y, forward):        
    # 기존 학습
    model = RandomForestRegressor(n_estimators=1000, min_samples_leaf=1)

    # model = XGBRegressor(max_delta_step=0.0, objective = 'reg:squarederror', nthread=-1,
    #                      n_estimators = 100,
    #                      colsample_bytree=0.9168,
    #                      learning_rate=0.1284,
    #                      max_depth=7,
    #                      min_child_weight=5,
    #                      subsample=0.4541)

    sfs = SequentialFeatureSelector(model,
                                    k_features=20, 
                                    forward=forward, # if forward = True then SFS otherwise SBS
                                    floating=True, 
                                    verbose=2,
                                    scoring='neg_mean_absolute_error',
                                    n_jobs=2,
                                    cv=5
                                    )

    #after applying sfs fit the data:
    # DataFrame 객체는 안됨. 넘파이객체만 됨
    sfs.fit(x.to_numpy(), y['pv_claim'].to_numpy())

    return sfs

In [5]:

# jobName = '201216_Train_split1'

# x, y = LibData.GetXYBase(jobName)
# x = LibData.GetMoneyness(x)
# x = LibData.GetAttainedAge(x)
# x = LibData.GetIrReduction(x)
# x = LibEE.GetX_Inter_EM_AA(x)
# x = LibEE.GetXEmbApplied(3, x, y, ['ProdCode', 'Inter_EM_AA'])
# scaler, x = LibUtils.ScaleStandard(x)

# x = x.sample(3000)
# y = y.iloc[x.index]


x = pd.read_csv('x.csv')
y = pd.read_csv('y.csv')

In [ ]:
sfbs = Selection(x, y, False)
print(sfbs.k_feature_names_)
LibUtils.ELoop()

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  23 out of  25 | elapsed:  9.6min remaining:   49.9s
[Parallel(n_jobs=8)]: Done  25 out of  25 | elapsed: 10.9min finished

[2021-01-05 02:51:52] Features: 24/10 -- score: -283.3286318798406[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  22 out of  24 | elapsed:  9.4min remaining:   51.0s
[Parallel(n_jobs=8)]: Done  24 out of  24 | elapsed:  9.4min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.5min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.5min finished

[2021-01-05 03:02:47] Features: 23/10 -- score: -279.3373254087228[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  20 out of  23 | elapsed:  8.9min remaining:  1.3min
[Parallel(n_jobs=8)]: Done  23

KeyboardInterrupt: 

In [ ]:
def GetSelectedFeatures(x, sfs, colCnt):
    cols = pd.DataFrame({ 'idx': range(len(x.columns)), 'colName': x.columns })
    selectedColIdxs = pd.DataFrame(sfs.subsets_[colCnt]['feature_idx'], columns=['idx'])
    selectedColIdxs = pd.merge(selectedColIdxs, cols, on='idx', how='left')
    del selectedColIdxs['idx']
    return selectedColIdxs['colName'].to_numpy()

for i in range(10, 16+1):
    res = GetSelectedFeatures(x, sfbs, i)
    print(res)



['Sex' 'Prem' 'PremYr' 'AccumPrem' 'StartAge' 'IR14' 'Moneyness'
 'AttainedAge' 'ee_ProdCode_01' 'ee_Inter_EM_AA_01']
['Sex' 'Prem' 'PremYr' 'AccumPrem' 'StartAge' 'FundAllo03' 'IR14'
 'Moneyness' 'AttainedAge' 'ee_ProdCode_01' 'ee_Inter_EM_AA_01']
['ElapsedMth' 'Sex' 'Prem' 'PremYr' 'AccumPrem' 'StartAge' 'FundAllo03'
 'IR14' 'Moneyness' 'AttainedAge' 'ee_ProdCode_01' 'ee_Inter_EM_AA_01']
['ElapsedMth' 'Sex' 'Prem' 'PremYr' 'AccumPrem' 'ContAmt' 'StartAge'
 'FundAllo03' 'IR14' 'Moneyness' 'AttainedAge' 'ee_ProdCode_01'
 'ee_Inter_EM_AA_01']
['ElapsedMth' 'Sex' 'Prem' 'PremYr' 'AccumPrem' 'ContAmt' 'StartAge'
 'FundAllo03' 'IR12' 'IR14' 'Moneyness' 'AttainedAge' 'ee_ProdCode_01'
 'ee_Inter_EM_AA_01']
['ElapsedMth' 'Sex' 'Prem' 'PremYr' 'AccumPrem' 'ContAmt' 'StartAge'
 'FundAllo03' 'IR09' 'IR12' 'IR14' 'Moneyness' 'AttainedAge'
 'ee_ProdCode_01' 'ee_Inter_EM_AA_01']
['ElapsedMth' 'Sex' 'Prem' 'PremYr' 'AccumPrem' 'ContAmt' 'StartAge'
 'FundAllo03' 'IR06' 'IR09' 'IR12' 'IR14' 'Moneyness